In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Dropout,MaxPooling1D
from tensorflow.keras.layers import Flatten,BatchNormalization,Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('vber_features.csv')

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.300781,0.706492,0.756853,0.735217,0.731014,0.715156,0.667529,0.673754,0.722173,0.748969,...,4.319263e-06,0.000003,0.000002,2.286677e-06,5.131694e-06,8.065748e-06,0.000005,2.270858e-06,1.642365e-07,neutral
1,0.333008,0.805021,0.840313,0.821007,0.822327,0.829499,0.720681,0.677058,0.727558,0.760922,...,1.136750e-04,0.000112,0.000114,1.147467e-04,1.090023e-04,1.177636e-04,0.000120,1.187017e-04,1.099673e-04,neutral
2,0.176693,0.640605,0.712837,0.743736,0.714797,0.701402,0.694569,0.663853,0.688527,0.742508,...,7.999367e-07,0.000001,0.000001,5.350822e-07,3.480662e-07,8.398366e-07,0.000001,4.287777e-07,2.878995e-08,neutral
3,0.271272,0.674966,0.723259,0.724594,0.681302,0.670643,0.674574,0.630036,0.680146,0.708276,...,6.998011e-06,0.000007,0.000007,6.999257e-06,1.217880e-05,9.449916e-06,0.000008,2.638513e-06,1.788902e-07,neutral
4,0.303828,0.751402,0.778547,0.806190,0.777157,0.766285,0.760808,0.658329,0.684237,0.730115,...,4.453990e-05,0.000044,0.000043,4.428105e-05,4.915007e-05,4.657156e-05,0.000044,3.983445e-05,3.691425e-05,neutral


In [4]:
len(df)

4320

In [5]:
df['labels'].value_counts()

calm        576
happy       576
sad         576
angry       576
fear        576
disgust     576
surprise    576
neutral     288
Name: labels, dtype: int64

In [6]:
X=df.drop('labels',axis=1)
y=df['labels']

In [7]:
scaler = StandardScaler()
X=scaler.fit_transform(X)

In [8]:
y=pd.get_dummies(y)

In [9]:
y

,angry,calm,disgust,fear,happy,neutral,sad,surprise
0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
4315,0,0,0,0,0,0,0,1
4316,0,0,0,0,0,0,0,1
4317,0,0,0,0,0,0,0,1
4318,0,0,0,0,0,0,0,1


In [10]:
x_train,x_test,y_train,y_test=train_test_split(X,y,shuffle=True,test_size=0.2,random_state=42)

In [11]:
x_train.shape

(3456, 162)

In [12]:
y_train.shape

(3456, 8)

In [13]:
x_test.shape

(864, 162)

In [14]:
y_test.shape

(864, 8)

In [15]:
y_test.head()

,angry,calm,disgust,fear,happy,neutral,sad,surprise
461,1,0,0,0,0,0,0,0
109,0,0,0,1,0,0,0,0
2295,0,0,1,0,0,0,0,0
2589,0,0,0,0,0,0,1,0
157,0,0,0,0,0,0,0,1


In [16]:
y_test.reset_index(drop = True,inplace=True)

In [17]:
y_train.reset_index(drop = True, inplace = True)

In [18]:
x_train=np.array(x_train).reshape([3456, 162,-1])

In [19]:
x_test=np.array(x_test).reshape([864, 162,-1])

### Conv1D

In [20]:
model=Sequential()
model.add(Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2,  padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(512, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2,  padding = 'same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Conv1D(1024, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2,  padding = 'same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=8, activation='softmax'))



In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 162, 256)          1024      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 81, 256)          0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 81, 256)          1024      
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 81, 512)           393728    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 41, 512)          0         
 1D)                                                             
                                                        

In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

rlrp = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=0, patience=2, min_lr=0.00000001)

In [25]:
history=model.fit(x_train,y_train, epochs=100,validation_data=(x_test, y_test), callbacks=[rlrp])

Epoch 1/100
108/108 [==============================] - 19s 73ms/step - loss: 3.8265 - accuracy: 0.1913 - val_loss: 2.0632 - val_accuracy: 0.1470 - lr: 0.0010
Epoch 2/100
108/108 [==============================] - 7s 67ms/step - loss: 2.0281 - accuracy: 0.2378 - val_loss: 2.0724 - val_accuracy: 0.1505 - lr: 0.0010
Epoch 3/100
108/108 [==============================] - 7s 62ms/step - loss: 1.8320 - accuracy: 0.3035 - val_loss: 1.9049 - val_accuracy: 0.2836 - lr: 0.0010
Epoch 4/100
108/108 [==============================] - 7s 61ms/step - loss: 1.6451 - accuracy: 0.3762 - val_loss: 1.7940 - val_accuracy: 0.2951 - lr: 0.0010
Epoch 5/100
108/108 [==============================] - 7s 63ms/step - loss: 1.5339 - accuracy: 0.4317 - val_loss: 1.5364 - val_accuracy: 0.4282 - lr: 0.0010
Epoch 6/100
108/108 [==============================] - 7s 64ms/step - loss: 1.4052 - accuracy: 0.4954 - val_loss: 2.1058 - val_accuracy: 0.3345 - lr: 0.0010
Epoch 7/100
108/108 [==============================] - 7s

108/108 [==============================] - 7s 60ms/step - loss: 0.0087 - accuracy: 0.9986 - val_loss: 2.9950 - val_accuracy: 0.8102 - lr: 7.8125e-06
Epoch 53/100
108/108 [==============================] - 7s 63ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 3.0419 - val_accuracy: 0.8113 - lr: 7.8125e-06
Epoch 54/100
108/108 [==============================] - 7s 69ms/step - loss: 0.0319 - accuracy: 0.9971 - val_loss: 3.0508 - val_accuracy: 0.8079 - lr: 7.8125e-06
Epoch 55/100
108/108 [==============================] - 6s 60ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 3.0895 - val_accuracy: 0.8079 - lr: 7.8125e-06
Epoch 56/100
108/108 [==============================] - 7s 60ms/step - loss: 0.0091 - accuracy: 0.9971 - val_loss: 3.0599 - val_accuracy: 0.8102 - lr: 3.9063e-06
Epoch 57/100
108/108 [==============================] - 7s 61ms/step - loss: 0.0180 - accuracy: 0.9971 - val_loss: 3.0785 - val_accuracy: 0.8102 - lr: 3.9063e-06
Epoch 58/100
108/108 [===================

In [26]:
model.evaluate(x_test,y_test)

27/27 [==============================] - 1s 18ms/step - loss: 3.0948 - accuracy: 0.8102


[3.0948312282562256, 0.8101851940155029]

In [41]:
x=x_test[4].reshape(1,162,-1)

In [42]:
y_pred = model.predict(x)

In [43]:
np.argmax(y_pred)

2

In [37]:
y_test

,angry,calm,disgust,fear,happy,neutral,sad,surprise
0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
859,0,0,1,0,0,0,0,0
860,1,0,0,0,0,0,0,0
861,0,0,1,0,0,0,0,0
862,0,1,0,0,0,0,0,0


In [ ]:
#model.save('vber_rav_conv1d_8206.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc="lower right")
plt.show()